In [214]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python:{}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python:3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
NLTK: 3.4.5
sklearn: 0.22.1
Pandas: 1.0.1
Numpy: 1.18.1


## 1. Load the Dataset

In [215]:
import pandas as pd
import numpy as np

#Load the dataset of sms messges

df = pd.read_table('SMSSpamCollection', header = None, encoding = 'utf-8')

In [216]:
# print useful information  about the data set

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [217]:
#check class distribution
classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


## 2. Preprocessing Data

In [218]:
#convert class labels to binary values, 0 - ham, 1 - spam

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [219]:
# store the SMS message data
text_messages = df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


## 2.1 Regular Expressions

Some common regular expression metacharacters - copied from wikipedia

^ Matches the starting position within the string. In line-based tools, it matches the starting position of any line.

. Matches any single character (many applications exclude newlines, and exactly which characters are considered newlines is flavor-, character-encoding-, and platform-specific, but it is safe to assume that the line feed character is included). Within POSIX bracket expressions, the dot character matches a literal dot. For example, a.c matches "abc", etc., but [a.c] matches only "a", ".", or "c".

[ ] A bracket expression. Matches a single character that is contained within the brackets. For example, [abc] matches "a", "b", or "c". [a-z] specifies a range which matches any lowercase letter from "a" to "z". These forms can be mixed: [abcx-z] matches "a", "b", "c", "x", "y", or "z", as does [a-cx-z]. The - character is treated as a literal character if it is the last or the first (after the ^, if present) character within the brackets: [abc-], [-abc]. Note that backslash escapes are not allowed. The ] character can be included in a bracket expression if it is the first (after the ^) character: []abc].

[^ ] Matches a single character that is not contained within the brackets. For example, [^abc] matches any character other than "a", "b", or "c". [^a-z] matches any single character that is not a lowercase letter from "a" to "z". Likewise, literal characters and ranges can be mixed.

$ Matches the ending position of the string or the position just before a string-ending newline. In line-based tools, it matches the ending position of any line.

( ) Defines a marked subexpression. The string matched within the parentheses can be recalled later (see the next entry, \n). A marked subexpression is also called a block or capturing group. BRE mode requires ( ).

\n Matches what the nth marked subexpression matched, where n is a digit from 1 to 9. This construct is vaguely defined in the POSIX.2 standard. Some tools allow referencing more than nine capturing groups.

* Matches the preceding element zero or more times. For example, abc matches "ac", "abc", "abbbc", etc. [xyz] matches "", "x", "y", "z", "zx", "zyx", "xyzzy", and so on. (ab)* matches "", "ab", "abab", "ababab", and so on.

{m,n} Matches the preceding element at least m and not more than n times. For example, a{3,5} matches only "aaa", "aaaa", and "aaaaa". This is not found in a few older instances of regexes. BRE mode requires {m,n}.

In [220]:
# regexlib.com

#use regular expression to replace email addresses, urls, phone numbers, other numbers, symbols

#replace email addresses with 'emailaddr'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailaddr')

#replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')

#replace money symbols with 'moneysymb'  - to get '£' pound sign press alt+156
processed = processed.str.replace(r'£|\$','moneysymb')

#replace 10 digit phone number to 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','phonenumbr')

#replace numbers with 'numbr'
processed = processed.str.replace('\d+(\.\d+)?','numbr')


In [221]:
#remove punctuation

processed = processed.str.replace('[^\w\d\s]', ' ')

#replace whitespace between  terms with  a single space
processed = processed.str.replace(r'\s+',' ')

#remove leading and training whitespace
processed = processed.str.replace(r'^\s+|\s+?$','')

In [222]:
# changing to the lower case  hello, Hello, HELLO to hello all the same
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [223]:
# remove stop words from text messages

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [224]:
#remove word stems using a Porter Stemmer

ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [225]:
print (processed)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri numbr wkli comp win fa cup final tk...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    numbrnd time tri numbr contact u u moneysymbnu...
5568                              ü b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: 1, Length: 5572, dtype: object


In [226]:
from nltk.tokenize import word_tokenize

#creating a bag of words model
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [227]:
# print the total number of words and also 15 most common words
print('Number of words{}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words6579
Most common words: [('numbr', 2648), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumbr', 303), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266)]


In [228]:
# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]

In [229]:
# define a find_feature function
def find_feature(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word]=(word in words)
    return features

#Lets see some examples

features = find_feature(processed[0])
for key, value in features.items():
    if value == True:
        print(keys)

go
go
go
go
go
go
go
go
go
go
go
go
go
go
go
go


In [230]:
processed[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [231]:
# find features for all the messages

messages = zip(processed, Y)

#define a seed  for reproductibility

seed = 1
np.random.seed = seed
##np.random.shuffle(messages)

#call find_features function for each sms messages

featuresets= [(find_feature(text), label) for (text, label) in messages]

In [232]:
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state = seed)

In [233]:
print('Training : {}'.format(len(training)))
print('Testing : {}'.format(len(testing)))


Training : 4179
Testing : 1393


## 3. Scikit-Learn Classifiers with NLTK

In [234]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [235]:
# Define model to train
names = ['K Nearest Neighbors','Decision Tree','Random Forest', 'Logistic Regression','SGD Classifier','Naive Bayes','SVM' ]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names,classifiers)


In [236]:
#wrap models in NLTK

from nltk.classify.scikitlearn import SklearnClassifier

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print('{}:Accuracy:{}'.format(name,accuracy))

K Nearest Neighbors:Accuracy:94.11342426417804
Decision Tree:Accuracy:97.5592246949031
Random Forest:Accuracy:98.85139985642498
Logistic Regression:Accuracy:98.27709978463747
SGD Classifier:Accuracy:98.63603732950466
Naive Bayes:Accuracy:98.63603732950466


In [237]:
#ensemble method - voting classifier

In [241]:
from sklearn.ensemble import VotingClassifier

names = ['K Nearest Neighbors','Decision Tree','Random Forest', 'Logistic Regression','SGD Classifier','Naive Bayes','SVM' ]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names,classifiers))

nltk_ensemble =  SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard',n_jobs = -1)) 
#hard voting and soft voting. when the classifiers are optimized you can go for soft voting. but when you need individually data then hard
#n_jobs = -1 means the classifiers will run parallel

nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_ensemble, testing)*100
print('Ensemble Method Accuracy:{}'.format(accuracy))

Ensemble Method Accuracy:98.7078248384781


In [243]:
#make class level prediction for testing set

txt_features, labels = list(zip(*testing)) 
prediction = nltk_ensemble.classify_many(txt_features)

In [247]:
#print a confusion matrix and a classification report 
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels,prediction),
    index = [['actual','actual'], ['ham','spam']],
    columns = [['predicted','predicted'],['ham','spam']])

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1208
           1       0.98      0.92      0.95       185

    accuracy                           0.99      1393
   macro avg       0.99      0.96      0.97      1393
weighted avg       0.99      0.99      0.99      1393



predicted     
                  ham spam
actual ham       1205    3
       spam        15  170